In [1]:
"""
Created on Sat Jan 23 20:27:10 2021

@author: Wasim
"""

import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 

In [2]:
data=pd.read_csv('model_data.csv')

In [3]:
data.shape

(887, 10)

In [4]:
data[data['Type']!='Apartments']

,Type,bedroom,bathroom,furnishing,Carpet Area (ft²),Maintenance (Monthly),facing,Total floor,state,Rent
15,Houses & Villas,4,4,Furnished,3000.00,0.0,East,1,Maharashtra,50000
17,Houses & Villas,2,1,Unfurnished,500.00,0.0,others,2,Maharashtra,7500
25,Builder Floors,2,2,Unfurnished,535.00,0.0,North-East,19,Maharashtra,14000
37,Builder Floors,1,1,Semi-Furnished,600.00,0.0,East,3,Maharashtra,6000
40,Houses & Villas,1,1,Semi-Furnished,580.00,100.0,East,1,Maharashtra,4500
...,...,...,...,...,...,...,...,...,...,...
878,Houses & Villas,1,1,Unfurnished,500.00,300.0,North,2,Tamil Nadu,6000
881,Houses & Villas,1,1,Semi-Furnished,400.00,500.0,North,2,Tamil Nadu,10000
883,Houses & Villas,2,1,Semi-Furnished,650.00,0.0,South,2,Tamil Nadu,12000
884,Houses & Villas,1,1,Unfurnished,185.88,0.0,others,0,Tamil Nadu,4000


In [5]:
data.state=data.state.apply(lambda x: x.strip())

In [6]:
def plot_scatter_chart(df,location):
    ap = df[(df.state==location) & (df.Type=='Apartments')]
    hv = df[(df.state==location) & (df.Type=='Houses & Villas')]
    bf = df[(df.state==location) & (df.Type=='Builder Floors')]
    #plt.rcParams['figure.figsize'] = (15,10)
    plt.scatter(ap['Carpet Area (ft²)'],ap['Rent'],color='blue',label='Apartments', s=50)
    plt.scatter(hv['Carpet Area (ft²)'],hv['Rent'],marker='+', color='green',label='Houses & Villas', s=50)
    plt.scatter(bf['Carpet Area (ft²)'],bf['Rent'],marker='x', color='red',label='others', s=50)
    plt.xlabel("Total Square Feet Area")
    plt.ylabel("Price (Lakh Indian Rupees)")
    plt.title(location)
    plt.legend()
    plt.savefig(location+'no_out.png', bbox_inches='tight')
    
    
plot_scatter_chart(df,"Karnataka")

NameError: name 'df' is not defined

In [ ]:
plot_scatter_chart(df,"Maharashtra")


In [ ]:
plot_scatter_chart(df,"Uttar Pradesh")

In [ ]:
plot_scatter_chart(df,"Tamil Nadu")

In [ ]:
plot_scatter_chart(df,"Kerala")

In [ ]:
data.Rent.describe()

In [ ]:
df.Rent.describe()

In [7]:
data[data.Rent>100000].shape

(55, 10)

In [8]:
df=data[data.Rent<100000]

In [9]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

In [10]:
num_pipeline=Pipeline([('std_scaler',StandardScaler())])

In [42]:
num_attribs=['Carpet Area (ft²)','Maintenance (Monthly)','Total floor','bedroom','bathroom']
cat_attribs=['Type','furnishing','facing','state']

In [ ]:
full_pipeline=ColumnTransformer([('num',num_pipeline,num_attribs),('cat',OneHotEncoder(),cat_attribs)])

In [11]:
def pipeline_transformer(data):
    
    num_attribs=['Carpet Area (ft²)','Maintenance (Monthly)','Total floor','bedroom','bathroom']
    cat_attribs=['Type','furnishing','facing','state']
    #num_attrs, num_pipeline = num_pipeline_transformer(data)
    num_pipeline=Pipeline([('std_scaler',StandardScaler())])
    full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
        ])
    full_pipeline.fit(data)
    return full_pipeline

In [12]:
full_pipeline = pipeline_transformer(df.drop(columns='Rent'))

In [13]:
prepared_df = full_pipeline.transform(df.drop(columns='Rent'))
print(len(prepared_df[0]))

26


In [14]:
X=prepared_df

In [15]:
len(X[0])

26

In [16]:
y=df['Rent'].values

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
from sklearn.linear_model import LinearRegression
lr_clf = LinearRegression()
lr_clf.fit(X_train,y_train)

LinearRegression()

In [20]:
lr_clf.score(X_train,y_train)

0.476066314407921

In [21]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [22]:

from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

cross_val_score(LinearRegression(), X, y, cv=cv)

array([ 0.2592901 ,  0.37380046, -1.86859429, -6.3716579 ,  0.37481703])

In [23]:
from sklearn.svm import SVR
regressor = SVR(kernel = 'rbf')
regressor.fit(X_train, y_train)

SVR()

In [24]:
y_pred=regressor.predict(X_test)

In [25]:
mean_absolute_error(y_test,y_pred)

5754.151737364948

In [26]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()

In [27]:
rf.fit(X_train, y_train)

RandomForestRegressor()

In [28]:
y_pred=rf.predict(X_test)

In [29]:
mean_absolute_error(y_test,y_pred)

5063.296933276304

In [30]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [int(x) for x in np.linspace(start = 5 , stop = 60, num = 10)] # returns 10 numbers 

max_features = ['auto', 'log2']

max_depth = [int(x) for x in np.linspace(3, 12, num = 3)] 

max_depth.append(None)

bootstrap = [True, False]

r_grid = {'n_estimators': n_estimators,

               'max_features': max_features,

               'max_depth': max_depth,

               'bootstrap': bootstrap}

print(r_grid)

{'n_estimators': [5, 11, 17, 23, 29, 35, 41, 47, 53, 60], 'max_features': ['auto', 'log2'], 'max_depth': [3, 7, 12, None], 'bootstrap': [True, False]}


In [31]:
rfr_random = RandomizedSearchCV(estimator=rf, param_distributions=r_grid, n_iter = 20, scoring='neg_mean_absolute_error', cv = 3, verbose=2, random_state=42, n_jobs=-1, return_train_score=True)

rfr_random.fit(X_train, y_train);

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    5.1s finished


In [32]:
print(rfr_random.best_params_)

{'n_estimators': 29, 'max_features': 'auto', 'max_depth': 7, 'bootstrap': True}


In [33]:
y_pred=rfr_random.predict(X_test)
mean_absolute_error(y_test,y_pred)

5115.225828710587

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
{'n_estimators': [35, 45,48,52], 'max_features' : ['auto', 'log2'], 
 'max_depth': [3, 6,10, None], 'bootstrap': [True,False]}
]

grid_search_forest = GridSearchCV(rf, param_grid, cv=10, scoring='neg_mean_squared_error')
grid_search_forest.fit(X_train, y_train)

In [ ]:
grid_search_forest.best_params_

In [ ]:
y_pred=grid_search_forest.predict(X_test)

In [ ]:
mean_absolute_error(y_test,y_pred)

In [34]:
from sklearn.ensemble import GradientBoostingRegressor

In [35]:
gbrt=GradientBoostingRegressor(max_depth=2,warm_start=True)

In [36]:
min_val_error=float('inf')
error_going_up=0

In [37]:
for n_estimators in range(1,120):
    gbrt.n_estimators=n_estimators
    gbrt.fit(X_train,y_train)
    y_pred=gbrt.predict(X_test)
    val_error=mean_absolute_error(y_test,y_pred)
    if val_error<min_val_error:
        min_val_error=val_error
        error_going_up=0
    else:
        error_going_up +=1
        if error_going_up==6:
            break #early stopping

In [38]:
min_val_error

4982.8898904544085

In [39]:
n_estimators

42

In [64]:

import pickle
pickl = {'model': gbrt}
pickle.dump( pickl, open( 'rent_model_file' + ".pkl", "wb" ) )

In [71]:
from sklearn.externals import joblib

# Save to file in the current working directory
joblib_file = "joblib_model.pkl"
joblib.dump(gbrt, joblib_file)

ImportError: cannot import name 'joblib' from 'sklearn.externals' (C:\Users\Wasim\anaconda3\lib\site-packages\sklearn\externals\__init__.py)

In [45]:

import json
columns = {
    'data_columns' : [col.lower() for col in labels]
}
with open("columns.json","w") as f:
    f.write(json.dumps(columns))

In [43]:
enc_cat_features = full_pipeline.named_transformers_['cat'].get_feature_names()
labels = np.concatenate([num_attribs, enc_cat_features])

In [44]:
labels

array(['Carpet Area (ft²)', 'Maintenance (Monthly)', 'Total floor',
       'bedroom', 'bathroom', 'x0_Apartments', 'x0_Builder Floors',
       'x0_Houses & Villas', 'x1_Furnished', 'x1_Semi-Furnished',
       'x1_Unfurnished', 'x2_East', 'x2_North', 'x2_North-East',
       'x2_North-West', 'x2_South', 'x2_South-East', 'x2_South-West',
       'x2_West', 'x2_others', 'x3_India', 'x3_Karnataka', 'x3_Kerala',
       'x3_Maharashtra', 'x3_Tamil Nadu', 'x3_Uttar Pradesh'],
      dtype=object)

In [ ]:
enc_cat_features = full_pipeline.named_transformers_['cat'].get_feature_names()

In [ ]:
len(enc_cat_features)


In [ ]:
len(X[0])

In [1]:
vehicle_config = {
    'Carpet Area (ft²)': [444, 666, 890],
    'Maintenance (Monthly)': [1555.0, 1600.0, 16500.5],
    'Total floor': [4, 2, 6],
    'bedroom': [1, 2, 2],
    'bathroom': [1, 2, 2],
    'Type': ['Houses & Villas', 'Houses & Villas', 'Apartments'],
    'furnishing':['Furnished','Unfurnished','Unfurnished'],
    'facing':['North','East','North-East'],
    'state': ['Tamil Nadu', 'Tamil Nadu', 'Maharashtra']
}

In [65]:
##loading the model from the saved file
import pickle
with open('rent_model_file.pkl', 'rb') as f_in:
    pickel_model = pickle.load(f_in)

In [70]:
predict_mpg(vehicle_config, pickel_model)

14
['x0_Apartments' 'x0_Builder Floors' 'x0_Houses & Villas' 'x1_Furnished'
 'x1_Semi-Furnished' 'x1_Unfurnished' 'x2_East' 'x2_North' 'x2_North-East'
 'x2_North-West' 'x2_South' 'x2_South-East' 'x2_South-West' 'x2_West'
 'x2_others' 'x3_India' 'x3_Karnataka' 'x3_Kerala' 'x3_Maharashtra'
 'x3_Tamil Nadu' 'x3_Uttar Pradesh']


AttributeError: 'dict' object has no attribute 'predict'

In [69]:
def predict_mpg(config, model):
    
    if type(config) == dict:
        df = pd.DataFrame(config)
    else:
        df = config

    pipeline = pipeline_transformer(df)
    prepared_df = pipeline.transform(df)
    print(len(prepared_df[0]))
    print(full_pipeline.named_transformers_['cat'].get_feature_names())
    y_pred = model.predict(prepared_df)
    return y_pred

In [6]:
import requests
url=' http://127.0.0.1:5000/'
r=requests.post(url,json=vehicle_config)
r.text.strip()

'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">\n<title>404 Not Found</title>\n<h1>Not Found</h1>\n<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>'